In [23]:
import os
import chess.pgn
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
import chess.engine
import random
from tqdm import tqdm

# Configuración del motor de ajedrez Stockfish
engine_path = r"C:\Users\marco\OneDrive\Ajedrez Computacional\stockfish-windows-x86-64-sse41-popcnt\stockfish\stockfish-windows-x86-64-sse41-popcnt.exe"
engine = chess.engine.SimpleEngine.popen_uci(engine_path)

# Función para leer un archivo PGN y cargar las partidas
def read_pgn_file(pgn_filename):
    games = []
    print(f"Leyendo archivo PGN: {pgn_filename}")
    try:
        with open(pgn_filename) as pgn:
            count = 0
            while True:
                game = chess.pgn.read_game(pgn)
                if game is None:
                    break
                games.append(game)
                count += 1
                if count % 500 == 0:
                    print(f"{count} partidas leídas...")
        print(f"Total de partidas leídas: {count}")
    except FileNotFoundError:
        print(f"Archivo {pgn_filename} no encontrado.")
    except Exception as e:
        print(f"Error al leer el archivo PGN: {str(e)}")
    return games

# Leer y cargar las partidas desde el archivo PGN existente
output_filename = r'C:\Users\marco\OneDrive\Ajedrez Computacional\merged_2019_2022_27.pgn-A3QEPXnBz2TgPJMQ\merged_2019_2022_27.pgn'
games = read_pgn_file(output_filename)

# Función para convertir partidas a formato de datos de entrenamiento con etiquetas reales
def game_to_training_data_with_labels(game, engine):
    board = game.board()
    data = []
    labels = []
    for move in game.mainline_moves():
        board.push(move)
        data.append(board_to_array(board))
        info = engine.analyse(board, chess.engine.Limit(time=0.05))  # Reducir tiempo de análisis a 0.05s
        eval_score = info["score"].relative.score(mate_score=10000)
        labels.append(eval_score if eval_score is not None else 0)
    return data, labels

# Función para convertir un tablero de ajedrez a un array numpy
def board_to_array(board):
    array = np.zeros((8, 8, 12), dtype=np.int8)
    piece_map = board.piece_map()
    for square, piece in piece_map.items():
        piece_type = piece.piece_type - 1
        color = int(piece.color)
        array[square // 8, square % 8, piece_type + (6 * color)] = 1
    return array

# Convertir partidas a formato de entrenamiento con etiquetas reales
print("Convirtiendo partidas a formato de entrenamiento con etiquetas reales...")
training_data = []
training_labels = []

for idx, game in enumerate(tqdm(games, desc="Procesando partidas", unit="partida")):
    data, labels = game_to_training_data_with_labels(game, engine)
    training_data.extend(data)
    training_labels.extend(labels)
    if (idx + 1) % 500 == 0:
        print(f"{idx + 1} partidas convertidas...")

# Preparar datos de entrenamiento
print("Preparando datos para el entrenamiento...")
X_train = np.array(training_data)
y_train = np.array(training_labels)

# Definir un modelo de red neuronal simple
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(8, 8, 12)),  # Capa densa con 64 neuronas y activación ReLU
    layers.Flatten(),  # Aplanar la salida de la capa anterior
    layers.Dense(64, activation='relu'),  # Otra capa densa con 64 neuronas y activación ReLU
    layers.Dense(1)  # Capa de salida con una sola neurona
])

# Compilar el modelo con el optimizador y la función de pérdida
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mae'])

# Callback para guardar el modelo después de cada epoch
###ESTE CALLBACK DA fALLOS NO LOGRO AVERIGUAr  PORQUE
checkpoint_filepath = r"C:\Users\marco\OneDrive\Ajedrez Computacional\modelo_ajedrez_epoch_{epoch:02d}.h5"
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    save_best_only=False,
    monitor='loss',
    verbose=1
)

# Entrenar el modelo con callback para guardar el progreso
print("Entrenando el modelo...")
history = model.fit(X_train, y_train, epochs=20, callbacks=[checkpoint_callback])

# Guardar el modelo entrenado final
final_model_filepath = r"C:\Users\marco\OneDrive\Ajedrez Computacional\modelo_ajedrez_final.h5"
model.save(final_model_filepath)

# Verificar que el archivo del modelo final ha sido guardado
if os.path.exists(final_model_filepath):
    print(f"El modelo ha sido guardado correctamente en: {final_model_filepath}")
else:
    print("Error: El modelo no ha sido guardado correctamente.")

# Cerrar el motor de ajedrez
engine.quit()




Leyendo archivo PGN: C:\Users\marco\OneDrive\Ajedrez Computacional\merged_2019_2022_27.pgn-A3QEPXnBz2TgPJMQ\merged_2019_2022_27.pgn
500 partidas leídas...
1000 partidas leídas...
1500 partidas leídas...
2000 partidas leídas...
2500 partidas leídas...
3000 partidas leídas...
3500 partidas leídas...
4000 partidas leídas...
4500 partidas leídas...
5000 partidas leídas...
5500 partidas leídas...
6000 partidas leídas...
6500 partidas leídas...
7000 partidas leídas...
7500 partidas leídas...
8000 partidas leídas...
8500 partidas leídas...
9000 partidas leídas...
9500 partidas leídas...
10000 partidas leídas...
10500 partidas leídas...
11000 partidas leídas...
11500 partidas leídas...
12000 partidas leídas...
Total de partidas leídas: 12081
Convirtiendo partidas a formato de entrenamiento con etiquetas reales...


Procesando partidas:   0%|                                             | 20/12081 [3:01:26<1823:42:26, 544.35s/partida]


TimeoutError: 